In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

In [ ]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from fastcore.all import *

import torch
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
class CFG:
    wfold = 0
    data_folder = '../input/sartorius-cell-instance-segmentation/'
    anno_folder = '../input/sartorius-annotations/'
    model_folder = '../input/sartorius-dataset/'
    model_arch = 'mask_rcnn_R_50_FPN_3x.yaml'
    nof_iters = 10000
    THRESHOLDS = [.15, .35, .55]
    MIN_PIXELS = [75, 150, 75]

# Functions

In [ ]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def get_masks(fn, predictor):
    im = cv2.imread(str(fn))
    pred = predictor(im)
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= CFG.THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
#    mat = []
    used = np.zeros(im.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= CFG.MIN_PIXELS[pred_class]: 
            used += mask
#            mat.append(mask)
            res.append(rle_encode(mask))
    return res

# def get_masks(fn, predictor):
#     im = cv2.imread(str(fn))
#     outputs = predictor(im)
#     pred_masks = outputs['instances'].pred_masks.cpu().numpy()
#     res = []
#     used = np.zeros(im.shape[:2], dtype=int) # to remove overlaps
#     for mask in pred_masks:
#         mask = mask * (1-used)
#         used += mask
#         res.append(rle_encode(mask))
#     return res


In [ ]:
# Taken from
def compute_iou(labels, y_pred):
    """
    Computes the IoU for instance labels and predictions.

    Args:
        labels (np array): Labels.
        y_pred (np array): predictions

    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    # Compute intersection between all objects
    intersection = np.histogram2d(
        labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
    )[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins=true_objects)[0]
    area_pred = np.histogram(y_pred, bins=pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    iou = intersection / union
    
    return iou[1:, 1:]  # exclude background

def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.

    Args:
        threshold (float): Threshold.
        iou (np array): IoU matrix.

    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn


def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred) for truth, pred in zip(truths, preds)]

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)

# Data

Load the weights from a previously fitted model - R50 architecture, 10k iterations,  MaP IoU=0.2554

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/' + CFG.model_arch))
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
cfg.MODEL.WEIGHTS = os.path.join(CFG.model_folder, 'model_f'+str(CFG.wfold)+'.pth')  
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
predictor = DefaultPredictor(cfg)


Get RLE for validation set, ground truth for validation set and RLE for the test set

In [ ]:
# RLE for ground truth
xdat = pd.read_csv('../input/sartorius-annotations/gt_fold.csv')
xdat = xdat.loc[xdat.fold == CFG.wfold][['id', 'annotation']].copy()

rle_gt = xdat['annotation'].copy()
ids_gt = xdat['id'].copy()

print(len(rle_gt), len(ids_gt))
valid_names = ids_gt.unique()


# RLE for validation set
ids_valid, rle_valid = [], []

for fn in valid_names:    
    rles = get_masks(CFG.data_folder + 'train/' + fn + '.png', predictor)
    for enc in rles:
        ids_valid.append(fn)
        rle_valid.append(enc)
        
print(len(rle_valid), len(ids_valid))


# RLE for test set
ids_test, rle_test = [] , []
test_names = [f[:-4] for f in os.listdir(CFG.data_folder + 'test')]

for fn in test_names:
    rles = get_masks(CFG.data_folder + 'test/' + fn + '.png', predictor)
    for enc in rles:
        ids_test.append(fn)
        rle_test.append(enc)

print(len(rle_test), len(ids_test))
  

# Evaluation 

Convert RLE for gt / validation into masks

In [ ]:
# translate to masks
masks_valid, masks_gt = [], []

for (ii,rl) in enumerate(rle_valid): 
    m = rle_decode(rl)
    masks_valid.append(m * ii)
    
for (ii,rl) in enumerate(rle_gt): 
    m = rle_decode(rl)
    masks_gt.append(m * ii)
    

Combine the masks per file

In [ ]:
# aggregate 
masks_valid_agg = pd.DataFrame()
masks_valid_agg['id'] = ids_valid
masks_valid_agg['masks'] = masks_valid
masks_valid_agg = masks_valid_agg.groupby('id').max().reset_index()

masks_gt_agg = pd.DataFrame()
masks_gt_agg['id'] = ids_gt
masks_gt_agg['masks'] = masks_gt
masks_gt_agg = masks_gt_agg.groupby('id').max().reset_index()

# plt.imshow(masks_valid_agg.masks[0])

Calculate the metric:

In [ ]:
# muh_iou = compute_iou(masks_gt_agg['masks'][0], masks_valid_agg['masks'][0])
# precision_at(0.5, muh_iou)
#iou_map(masks_gt_agg['masks'] , masks_gt_agg['masks'], verbose=1)  # This should score 1

In [ ]:
iou_map(masks_gt_agg['masks'] , masks_valid_agg['masks'], verbose=1) 

# Submission

In [ ]:
pd.DataFrame({'id': ids_test, 'predicted': rle_test}).to_csv('submission.csv', index=False)